# Testing 123

In [1]:
# imports
import os
import glob
import numpy as np
import subprocess 
from pkg_resources import resource_filename

from importlib import reload

import pandas
import h5py
from skimage.restoration import inpaint 

from sklearn.utils import shuffle

from ulmo import io as ulmo_io

from ulmo.ssl.my_util import Params, option_preprocess
from ulmo.ssl import latents_extraction

from functools import partial
from concurrent.futures import ProcessPoolExecutor
import subprocess
from tqdm import tqdm

from tqdm.auto import trange

import torch
import tqdm

from ulmo.utils import HDF5Dataset

from ulmo.ssl.my_util import Params, option_preprocess
from ulmo.ssl.my_util import modis_loader, set_model
from ulmo.ssl.my_util import train_modis


from IPython import embed


# Refactoring

In [2]:
model_path = './'
model_name = "last.pth"

opt_file = os.path.join(resource_filename('ulmo', 'runs'),
                        'SSL', 'First','experiments', 
                        'base_modis_model', 'opts.json')
opt = Params(opt_file)
opt = option_preprocess(opt)

# Load the data
print("Grabbing MODIS")
#modis_dataset_path = "s3://modis-l2/PreProc/MODIS_R2019_2010_95clear_128x128_preproc_std.h5"
#base_file = os.path.basename(modis_dataset_path)
#if not os.path.isfile(base_file):
#    ulmo_io.download_file_from_s3(base_file, modis_dataset_path)
modis_dataset_path = "/data/Projects/Oceanography/AI/OOD/SST/MODIS_L2/PreProc/MODIS_R2019_2010_95clear_128x128_preproc_std.h5"

Grabbing MODIS


In [3]:

save_path = './experiments/modis_latents/'
if not os.path.isdir(save_path):
    os.makedirs(save_path)

model_path_title = os.path.join(model_path, model_name)
model_name = model_name.split('.')[0]
latents_path = os.path.join(save_path, f'MODIS_2010_valid_{model_name}.h5')
save_key = 'modis_latents'


# Run me

In [47]:
reload(latents_extraction)
#latents_extraction.model_latents_extract(opt, dataset_train, 
#                                         model_path_title, latents_path, 
#                                         save_key)

<module 'ulmo.ssl.latents_extraction' from '/tank/xavier/Oceanography/Python/ulmo/ulmo/ssl/latents_extraction.py'>

In [52]:
dset, loader = latents_extraction.build_loader(modis_dataset_path, 'train')

## Load model

In [9]:
using_gpu=False
model_path = model_path_title
remove_module=True
model, _ = set_model(opt, cuda_use=using_gpu)
if not using_gpu:
    model_dict = torch.load(model_path, map_location=torch.device('cpu'))
else:
    model_dict = torch.load(model_path)

if remove_module:
    new_dict = {}
    for key in model_dict['model'].keys():
        new_dict[key.replace('module.','')] = model_dict['model'][key]
    model.load_state_dict(new_dict)
else:
    model.load_state_dict(model_dict['model'])
print("Model loaded")

Model loaded


## Test me

In [54]:
with torch.no_grad():
    #for data in tqdm(loader, total=len(loader), unit='batch', desc='Computing log probs'):
    #    tmp = latents_extraction.calc_latent(model,data[0], False)
    tmp = [latents_extraction.calc_latent(model,data[0], False) for data in tqdm.tqdm(loader, total=len(loader), unit='batch', desc='Computing log probs')]

Computing log probs:   1%|          | 1368/135712 [06:47<11:16:32,  3.31batch/s]

KeyboardInterrupt: 

----

# Check output

## New

In [2]:
orig_disk = h5py.File('MODIS_orig_2010_train_last.h5', 'r')

In [3]:
disk_latents = orig_disk['modis_latents'][:]

In [4]:
disk_latents.shape

(100, 128)

## s3

In [5]:
latents_last_file = 's3://modis-l2/modis_latents_simclr/modis_latents_last.h5'
latents_last = ulmo_io.load_to_bytes(latents_last_file)
#
with h5py.File(latents_last, 'r') as file:
    print(file.keys())
    latents_last = file['modis_latents'][:]

<KeysViewHDF5 ['modis_latents']>


In [6]:
disk_latents[0,:] - latents_last[0,:]

array([ 0.00105257,  0.06177367, -0.02333895, -0.05875482, -0.01694185,
        0.00194306, -0.02446658, -0.04061244, -0.02464695,  0.11207053,
       -0.03562526, -0.23334265, -0.07392522, -0.14564475,  0.09883755,
        0.01244625,  0.09143987, -0.11838783,  0.25236827, -0.11037388,
       -0.07979135, -0.21131252,  0.20098463,  0.23648924, -0.03195838,
        0.04339853, -0.05101895,  0.13686971, -0.01488656,  0.29862443,
        0.005424  , -0.09710206,  0.02991411,  0.07805967, -0.077572  ,
       -0.00156219,  0.09732082, -0.21347386,  0.07724583,  0.03751778,
       -0.11285279, -0.37476546, -0.11896938,  0.00438425,  0.05728772,
       -0.04825673, -0.01616359,  0.13115562,  0.02367653,  0.11729351,
        0.19191825, -0.03863501,  0.06538799,  0.05340741, -0.05474619,
       -0.0257122 ,  0.09053737,  0.08282354,  0.07845354,  0.00918123,
        0.13219628, -0.0861268 , -0.17026547, -0.15054172, -0.2173146 ,
        0.10457973, -0.05036297,  0.02025899,  0.19367157, -0.14

## Checking agains Erdong's approach

In [19]:
with h5py.File('orig_latents.h5', 'r') as file:
    print(file.keys())
    latents_orig = file['modis_latents'][:]
latents_orig.shape

<KeysViewHDF5 ['modis_latents']>


(100, 128)

In [20]:
disk_latents[0,:] - latents_orig[0,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [18]:
latents_orig.shape

(128,)

----

# Debug set

In [7]:
orig_file = 'MODIS_2010_95clear_128x128_inpaintT_preproc_0.8valid.h5'

In [8]:
hf_orig = h5py.File(orig_file, 'r')
orig_train = hf_orig['train'][:]

In [9]:
orig_train.shape

(134162, 1, 64, 64)

## Save a subset

In [10]:
with h5py.File('test_orig.h5', 'w') as file:
    file.create_dataset('train', data=orig_train[0:100,...])

In [11]:
tmp = h5py.File('test_orig.h5', 'r')
tmp['train'].shape

(100, 1, 64, 64)